In [ ]:
# 필요한 라이브러리 임포트
import os                             # 파일 및 디렉토리 작업을 위한 모듈
import torch                          # PyTorch 딥러닝 프레임워크
import torch.nn as nn                 # 신경망 관련 모듈
import torch.optim as optim           # 최적화 알고리즘 모듈
from PIL import Image                 # 이미지 처리 모듈 (Pillow)
import shutil                         # 파일 및 폴더 복사/삭제 등을 위한 고급 파일 처리 모듈
from torchvision import transforms, models  # 이미지 변환 및 사전학습 모델
from torch.utils.data import Dataset, DataLoader  # 커스텀 데이터셋 및 배치 로더
import matplotlib.pyplot as plt       # 그래프 시각화
from IPython.display import clear_output  # Jupyter Notebook에서 출력 갱신용

# 디바이스 설정: GPU가 사용 가능한 경우 'cuda', 아니면 'cpu'를 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 삭제할 폴더 경로 지정
folder_path = "checkpoints/train/weights"

# 해당 폴더가 존재하는지 확인
if os.path.exists(folder_path):
    # 폴더 안의 모든 파일과 하위 폴더를 순회
    for filename in os.listdir(folder_path):
        # 각 파일 또는 폴더의 전체 경로를 생성
        file_path = os.path.join(folder_path, filename)
        try:
            # 해당 경로가 '파일' 또는 '심볼릭 링크'인 경우 삭제
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # 파일 또는 링크 삭제
                print(f"파일 '{file_path}'이 삭제되었습니다.")
            # 해당 경로가 '디렉토리'인 경우 재귀적으로 삭제
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # 폴더 전체 삭제
                print(f"폴더 '{file_path}'이 삭제되었습니다.")
        except Exception as e:
            # 삭제 중 오류가 발생한 경우 에러 메시지를 출력
            print(f"'{file_path}'를 삭제하는 중 에러가 발생했습니다: {e}")
else:
    # 폴더가 존재하지 않는 경우 안내 메시지 출력
    print(f"폴더 '{folder_path}'가 존재하지 않습니다.")


In [ ]:
# 하이퍼파라미터 설정
EPOCH = 20        # 총 학습 에포크 수
pre_epoch = 0      # 시작 에포크 (재학습 시 사용)
BATCH_SIZE = 16    # 배치 크기
LR = 0.01          # 학습률
TRAIN_DATA_PERCENT = 0.8  # 훈련 데이터 비율

In [ ]:
# PyTorch의 Dataset 클래스를 상속하여 CustomDataset 정의
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir                    # 데이터셋의 루트 폴더 경로
        self.transform = transform                  # 이미지에 적용할 변환(transform) 함수
        # 클래스 이름 리스트를 오름차순으로 정렬하여 저장
        self.classes = sorted([
            d for d in os.listdir(root_dir)
            if os.path.isdir(os.path.join(root_dir, d))  # 폴더만 추출
        ])
        # 클래스 이름을 숫자 인덱스로 매핑 (예: {'cat': 0, 'dog': 1})
        self.class_to_idx = {
            cls_name: idx for idx, cls_name in enumerate(self.classes)
        }
        # 모든 이미지 경로와 라벨을 불러오는 함수 호출
        self.data = self._load_data()

    def _load_data(self):
        data = []
        for cls in self.classes:
            class_path = os.path.join(self.root_dir, cls)  # 클래스별 디렉토리 경로
            for img_name in os.listdir(class_path):        # 클래스 폴더 안의 모든 이미지 파일
                img_path = os.path.join(class_path, img_name)
                label = self.class_to_idx[cls]              # 클래스 이름을 숫자로 매핑
                data.append((img_path, label))              # (이미지 경로, 라벨) 튜플 저장
        return data

    def __len__(self):
        return len(self.data)  # 전체 데이터 개수 반환

    def __getitem__(self, idx):
        img_path, label = self.data[idx]                    # 인덱스에 해당하는 이미지 경로와 라벨
        image = Image.open(img_path).convert('RGB')         # 이미지 파일 열고 RGB로 변환
        if self.transform:                                  # transform이 설정되어 있으면 적용
            image = self.transform(image)
        return image, label                                 # 이미지와 라벨 반환


# 데이터 전처리 설정: 이미지를 224x224 크기로 리사이즈하고 텐서로 변환
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # 입력 크기를 사전학습 모델 크기에 맞춤
    transforms.ToTensor()            # [0, 1] 범위의 Tensor로 변환
])

# 전체 데이터셋 로드 (예: 'data' 폴더 구조는 data/class1/*.jpg, data/class2/*.jpg ...)
dataset = CustomDataset(root_dir="data", transform=transform)

# 전체 데이터셋에서 훈련/테스트 비율 설정
# 예: TRAIN_DATA_PERCENT = 0.8이라면 훈련 80%, 테스트 20%
train_size = int(TRAIN_DATA_PERCENT * len(dataset))
test_size = len(dataset) - train_size

# 데이터셋을 무작위로 훈련 세트와 테스트 세트로 분할
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# DataLoader는 데이터를 배치 단위로 불러오고, 멀티 스레딩(num_workers)을 활용
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# 훈련/테스트 데이터셋의 크기 출력
print(f"훈련 데이터 수: {len(train_dataset)}, 테스트 데이터 수: {len(test_dataset)}")
    